In [4]:
# Import the necessary functions
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance
import json

# Define the product query
query = "iphone16promax"

# Fetch prices from Croma
croma_prices = fetch_prices_from_croma(query)
print("Croma Prices:")
print(json.dumps(croma_prices, indent=4))

# Fetch prices from Reliance Digital
reliance_prices = fetch_prices_from_reliance(query)
print("Reliance Digital Prices:")
print(json.dumps(reliance_prices, indent=4))


Croma Prices:
[
    {
        "name": "GRIPP Duce Tempered Glass & Polycarbonate, Silicone Back Cover Combo for Apple iPhone 16 Pro Max (MagSafe Charging Support, Transparent)",
        "url": "https://www.croma.com/gripp-duce-tempered-glass-polycarbonate-silicone-back-cover-combo-for-apple-iphone-16-pro-max-magsafe-charging-support-transparent-/p/310637",
        "price": 999.0
    },
    {
        "name": "Dr. Vaku Tempered Glass, TPU Back Cover & Camera Lens Protector for Apple iPhone 16 Pro Max (Supports Wireless Charging)",
        "url": "https://www.croma.com/dr-vaku-tempered-glass-tpu-back-cover-camera-lens-protector-for-apple-iphone-16-pro-max-supports-wireless-charging-/p/310888",
        "price": 1299.0
    },
    {
        "name": "GRIPP Duce TPU Plus PC MagSafe Case, Tempered Glass and Camera Lens Protector Combo for Apple iPhone 16 Pro Max (Supports Wireless Charging, Clear)",
        "url": "https://www.croma.com/gripp-duce-tpu-plus-pc-magsafe-case-tempered-glass-and-cam

/Users/priya/miniconda3/envs/agentenv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.reliancedigital.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Reliance Digital Prices:
[
    {
        "name": "Neopack 3-in-1, 15 W Wireless Magnetic Charging Docking Station for iPhone, Airpods and Apple Watch, upto 15 W for iPhone/5 W for Airpods & 3 W for Apple Watch, Quick Heat Disspation, Over Charge, Circuit & Overnight Charge Protection, Hands free, WMD3IN1",
        "url": "https://www.reliancedigital.in/neopack-3-in-1-15-w-wireless-magnetic-charging-docking-station-for-iphone-airpods-and-apple-watch-upto-15-w-for-iphone-5-w-for-airpods-3-w-for-apple-watch-quick-heat-disspation-over-charge-circuit-overnight-charge-protection-hands-free-wmd3in1/p/493178403",
        "price": "1,899.00"
    },
    {
        "name": "Apple HomePod (2nd Generation), Intelligent Assistant and Voice Recognition, Temperature and Humidity Sensor, High-excursion Woofer, White",
        "url": "https://www.reliancedigital.in/apple-homepod-2nd-generation-intelligent-assistant-and-voice-recognition-temperature-and-humidity-sensor-high-excursion-woofer-white/p/493666